In [6]:
import sys
import torch
import os

In [7]:
os.chdir("..")

In [8]:
!ls

README.md	  jupyter.o7285616  jupyter.o7437985	src
checkpoints	  jupyter.o7288049  logs		training
data		  jupyter.o7301470  notebooks		translations
jupyter.o7250076  jupyter.o7315071  out.txt		venv
jupyter.o7272725  jupyter.o7333208  prose_translations	vt-tr.o7146959
jupyter.o7277176  jupyter.o7341731  requirements.txt	vt-tr.o7209762
jupyter.o7280433  jupyter.o7343014  runs		vt-tr.o7242335
jupyter.o7284871  jupyter.o7346884  scripts


In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
from src.data_utils.batch import rebatch
from src.data_utils.data import get_training_iterators
from src.model.loss_optim import MultiGPULossCompute, SimpleLossCompute
from src.model.model import make_model, NoamOpt, LabelSmoothing, translate_sentence
from src.utils.utils import get_tokenizer

In [11]:
tok = get_tokenizer("tr")

In [12]:
train_iter, valid_iter, test_iter, train_idx, dev_idx, test_idx = get_training_iterators("tur", batch_size=3000)

/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/iterator.py:48: UserWarning: MyIterator class will be retired soon and moved to torchtext.l

In [13]:
# mini dev set
with open("data/tr/tur.dev.tgt", encoding="utf-8") as infile:
    toystrings = [x.strip() for x in infile.readlines()[:20]]

In [14]:
toyset = [torch.LongTensor([1] + tok.Encode(x) + [2])  for x in toystrings]
toyset = torch.nn.utils.rnn.pad_sequence(sequences=toyset, padding_value=3)

In [15]:
toyset

tensor([[    1,     1,     1,  ...,     1,     1,     1],
        [ 5605,     8,  1330,  ...,     8,   771,  2804],
        [27861,  2475, 10284,  ...,  3987,  5057, 11694],
        ...,
        [    3,     3,     3,  ...,     3,     3,     3],
        [    3,     3,     3,  ...,     3,     3,     3],
        [    3,     3,     3,  ...,     3,     3,     3]])

Two critics:
- Input related to output or not
- Classifier into poetry, prose, generated, scrambled poetry

One word/token selector:
- Choose tokens from input sequence to use for topic
- 

In [16]:
from torchtext import data
import torchtext as tt
from src.data_utils.batch import MyIterator
from src.model.model import batch_size_val

def each_line(fname):
    c = 0
    lines = []
    with open(fname, "r", encoding="utf-8") as infile:
        for line in infile:
            if line.count(" ") > 200 or line.count(" ") < 10:
                continue
            lines.append(line.strip())
            c += 1
            if c >= 2000000: 
                break
    return lines

def make_iter(lines, tokenizer, batch_size=256):
    
    def tok(seq):
        return tokenizer.EncodeAsIds(seq)

    field = data.Field(tokenize=tok, init_token=1, eos_token=2, pad_token=3, use_vocab=False)
    #ds = data.TabularDataset(fpath, "tsv", [("src", field)], skip_header=True)

    examples = [tt.data.Example.fromdict({"src": x}, {"src": ("src", field)}) for x in lines]
    ds = tt.data.Dataset(examples, {"src": field})
    iter = MyIterator(ds, batch_size=batch_size, device="cpu",
                             repeat=False, sort_key=lambda x: len(x.src),
                             batch_size_fn=batch_size_val, train=False, sort=True)

    return iter




In [17]:
prose_iter = make_iter(each_line("data/tr/prose/prose_gan.txt"), tok, batch_size=3000)


/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [18]:
import random

to_scramble = each_line("data/tr/tur.train.tgt")
scrambled = []
for poem in to_scramble:
    new_poem = poem.split("¬")
    random.shuffle(new_poem)
    scrambled.append("¬".join(new_poem))

In [19]:
scrambled_iter = make_iter(scrambled, tok, batch_size=3000)

In [20]:
import copy
from src.model.model import MultiHeadedAttention, PositionwiseFeedForward, \
                    PositionalEncoding, Encoder, EncoderLayer, Generator, Embeddings
import torch.nn as nn

class Critic(nn.Module):

    def __init__(self, encoder, src_embed, generator):
        super(Critic, self).__init__()
        self.encoder = encoder
        self.src_embed = src_embed
        self.generator = generator
        self.steps = 0

        
    def forward(self, x, mask):
        """Pass the input (and mask) through each layer in turn."""
        x = self.src_embed(x)
        for layer in self.encoder.layers:
            x = layer(x, mask)
        return self.encoder.norm(x)    


def make_critic(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    """Helper: Construct a model from hyper-parameters."""
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    generator = Generator(d_model, tgt_vocab)
    embed = nn.Sequential(Embeddings(d_model, src_vocab), c(position))
    encoder = Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N)
    critic = Critic(encoder, embed, generator)
    
    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in critic.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)

    return critic


In [21]:
ntokens = 32000
enc_dec = make_model(ntokens, ntokens, N=6).to(device)
token_selector = make_critic(ntokens, 2, N=2).to(device)
style_critic = make_critic(ntokens, 4, N=2).to(device)
relevance_critic = make_critic(ntokens + 1, 1, N=2).to(device)

/auto/plzen1/home/memduh/versetorch/src/model/model.py:264: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:39: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [22]:
from torch.autograd import Variable
import numpy as np
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


def prep_tensors( src, trg, pad=3):
    src_mask = (src != pad).unsqueeze(-2)
    trg_in = trg[:, :-1]
    trg_y = trg[:, 1:]
    trg_mask = make_std_mask(trg_in, pad)
    return src, trg_y, src_mask, trg_mask

def make_std_mask(tgt, pad):
    """Create a mask to hide padding and future words."""
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(
        subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
    return tgt_mask


In [23]:
def get_dae_input(tgt, token_selector):
    select_prob_embeds = token_selector.forward(tgt.to(device), 
                                         (tgt != 3).unsqueeze(-2).to(device))
    select_prob = token_selector.generator(select_prob_embeds)
    select_indices = torch.max(select_prob, dim=2).indices.type(torch.ByteTensor)
    dae_list = []
    for ind, row in zip(select_indices, tgt):
        dae_list.append(torch.masked_select(row, ind)[:15])
    dae_input = torch.nn.utils.rnn.pad_sequence(dae_list, batch_first=False, padding_value=3)
    return dae_input

In [24]:
rebatched = (rebatch(3, b) for b in train_iter)

In [25]:
torch


<module 'torch' from '/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torch/__init__.py'>

In [26]:
from src.model.adafactor import Adafactor

#enc_dec_opt = NoamOpt(enc_dec.src_embed[0].d_model, 1, 2000,
#                        torch.optim.Adam(enc_dec.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))
enc_dec_opt = Adafactor(enc_dec.parameters())

style_criterion = nn.BCELoss()
relevance_criterion = nn.BCELoss()

token_optim = Adafactor(token_selector.parameters())
style_optim = Adafactor(style_critic.parameters())
rel_optim = Adafactor(relevance_critic.parameters())

In [27]:
relevance_criterion = torch.nn.BCEWithLogitsLoss()

In [28]:
def get_relevance_input(dae_input, tgt):
    mid_point = torch.ones((tgt.shape[0], 1), dtype=torch.long) * ntokens
    return torch.cat((dae_input, mid_point.to(device), tgt), dim=1)


In [29]:
accumulation_steps = 8

In [30]:
# get validation iterator


def validate_batch(model, src, max_len=256, start_symbol=1, end_symbol=2):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    src_mask = (src != 3).unsqueeze(-2)
    memory = model.encode(src.to(device), src_mask.to(device))
    ys = torch.ones(src.shape[0], 1).fill_(start_symbol).type_as(src.data).to(device)
    finished = torch.zeros((src.shape[0], 1))
    for i in range(max_len-1):
        out = model.decode(memory, src_mask,
                           Variable(ys).to(device),
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)).to(device))
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim = 1)
        # next_word = next_word.data_utils[0]
        unsqueezed = next_word.unsqueeze(1)
        for c, token in enumerate(unsqueezed):
            if token == end_symbol:
                finished[c] = 1
        if sum(finished) >= src.shape[0]:
            break
        ys = torch.cat([ys, unsqueezed], dim=1)
                        # torch.ones(src.shape[0], 1).type_as(src.data_utils).fill_(next_word).to(device)], dim=1)
    return ys


def validate(model, selector, iterator):
    pass

In [31]:

label_smoothing = LabelSmoothing(size=32000, padding_idx=3, smoothing=0.1)

/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torch/nn/_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [32]:
[tok.Decode(x.tolist()) for x in toyset.transpose(0, 1)]

['ayrılığın sınırlı ışığında¬ışıldar gözler¬dar gelir dünya¬sıkıcı karanlıktan¬çıkış başlar¬sevgi akla gelince¬ışık küçücüktür¬sevgi karanlığında¬sonsuzluğu hatırlatır insana¬ışık kandırıcıdır belki¬ayrılığı sevenler için¬orada yuvarlak bir masa¬üç beş sigara biraz da çay¬sevgi karanlığında herşey¬coşkuyla¬zaten aranan da o değil mi¬sınırlı değilmiş hiçbir şey',
 'Şair sözün gösterer,¬Gözel gözün gösterer.¬Payız özün gösterer,¬Saralmış yarpağıyla.¬Uzaqda quzey durar,¬Şa ⁇ ta qelbi hey vurar.¬Önümde muzey qurar,¬Heyat çılpaqlığıyla.¬Yel eser deyinerek,¬ ⁇ um-paltar geyinerek,¬Fe ⁇ r eder öyünərek¬Insan torpaqlığıyla.',
 'Ey yakarış¬Ey uyanış¬Ey sevgi¬Ey aşk¬Ey arayış¬Ey tükeniş¬Ey özleyiş¬Uyanın artık¬O derin uykulardan¬Ve devam edin¬Katıksızca aramaya¬Aşkı,sevdayı,¬Yakarışı,uyanışı¬Arayışı,tükenişi¬Özleyişi,özlemi',
 'ben ölsemde aşkım cümle alemde¬kulakdan kulaga yayılır belki¬adımız gecer her bir kalemde¬gönül nikahımız kıyılır belki¬sevdik sevildik ölürcesine¬başını sonunu bilircesi

In [33]:
import time


In [34]:
soft = torch.nn.Softmax(dim=1)
start = time.time()
all_tokens = 0
for c, (poetry_batch, prose_batch, scrambled_batch) in enumerate(zip(rebatched, prose_iter, scrambled_iter)):
    all_tokens += poetry_batch.ntokens
    tgt, tgt_mask = poetry_batch.trg.to(device), poetry_batch.trg_mask.to(device)
    # classify tokens, get the first 15 tokens selected.
    dae_input = get_dae_input(poetry_batch.trg, token_selector).transpose(0,1).to(device)
    # create src and src mask from selected tokens
    dae_input_mask = (dae_input != 3).unsqueeze(-2)
    
    # get output of poetry generator
    output_embeds = enc_dec.forward(dae_input, tgt, dae_input_mask, tgt_mask)
    output = enc_dec.generator(output_embeds)
    reconstruction_loss = label_smoothing(output.contiguous().view(-1, output.size(-1)),
                             poetry_batch.trg_y.to(device).contiguous().view(-1)) / poetry_batch.ntokens
    reconstruction_loss.backward()
    
    if c % accumulation_steps == 0 and c > 0:
        token_optim.step() # I don't want GAN loss touching this
        enc_dec_opt.step()
        enc_dec_opt.zero_grad()
        token_optim.zero_grad()

    
    if c % 100  == 0:
        print("Reconstruction loss:", reconstruction_loss)
        print(all_tokens / (time.time() - start), "tokens processed per second.")
        #print(torch.mean(gen_scores[:, 1]))
        if c% 500 == 0:
            val_src = get_dae_input(toyset.transpose(0, 1), token_selector).transpose(0,1).to(device)
            validated = validate_batch(enc_dec, val_src)
            print([tok.Decode(x.tolist()) for x in validated])
        

Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.


/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/auto/praha1/memduh/versetorch/venv/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: masked_select received a mask with dtype torch.uint8, this behavior is now deprecated,please use a mask with dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorAdvancedIndexing.cpp:733.)
  


Reconstruction loss: tensor(9.0428, device='cuda:0', grad_fn=<DivBackward0>)
tensor(391.9535) tokens processed per second.
['omzuna serpiştirVekilBeyoğluSaçlarındaDolmuş boyunaAnılarım omzunaCanımsın omzuna omzunaYarına maşaYarınaYarına koparılmışelinde beynimdeYarınaYarına kucaklar solda omzunalüğeYarına omzuna omzuna omzunaSızı omzuna kefereÖzünYarına çektirgıt beynimde eksilmez omzunaYarına artırırYarına mat yasaklıYarınaculuk çektirYarınağımızda yasaklı ....¬ omzuna omzunaCanımsınCanımsıngıtSızı omzuna omzunaÖzünCanımsınCanımsınCanımsın burdanSızı yayıCanımsın düşmanCanımsınYarınaCanımsın eksilmez omzuna önünü omzuna artırır omzuna omzuna omzunaYarına aklımda ....¬gıtCanımsınMalYarınaelindegıt omzuna boyuna kucaklar eksilmezYarına omzunalüğeəyəMalSızıYarınaGökyüzüSızı omzuna düşmanelinde ....¬Mal ....¬ eksilmez dikkatliYarınaSızılüğe omzuna bulmuşumCanımsın solda eksilmezYarınaSızılüğe malik kucaklar omzunaMal omzuna kucaklarlüğeMalYarına dikkatli çektir omzunaSızıYarına omzunaYarı

Reconstruction loss: tensor(8.2939, device='cuda:0', grad_fn=<DivBackward0>)
tensor(5812.0264) tokens processed per second.
Reconstruction loss: tensor(7.6751, device='cuda:0', grad_fn=<DivBackward0>)
tensor(7229.4761) tokens processed per second.
Reconstruction loss: tensor(7.4471, device='cuda:0', grad_fn=<DivBackward0>)
tensor(7882.6802) tokens processed per second.
Reconstruction loss: tensor(7.0039, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8236.6963) tokens processed per second.
Reconstruction loss: tensor(6.9953, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8472.5820) tokens processed per second.
['¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬', '¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬¬

Reconstruction loss: tensor(6.7420, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8631.1680) tokens processed per second.
Reconstruction loss: tensor(6.8023, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8695.9424) tokens processed per second.
Reconstruction loss: tensor(6.6656, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8757.8271) tokens processed per second.
Reconstruction loss: tensor(6.8302, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8809.2568) tokens processed per second.
Reconstruction loss: tensor(6.7222, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8848.0889) tokens processed per second.
['Bir bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bir¬Bi

...

Reconstruction loss: tensor(5.0443, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8726.6621) tokens processed per second.
Reconstruction loss: tensor(5.2284, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8729.3809) tokens processed per second.
Reconstruction loss: tensor(6.0388, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8732.2314) tokens processed per second.
Reconstruction loss: tensor(5.4637, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8735.0166) tokens processed per second.
Reconstruction loss: tensor(5.1489, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8737.6641) tokens processed per second.
[' ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’’’ ⁇ ’ ⁇ ’’ ⁇ ’ ⁇ ’ ⁇ ’’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ ⁇ ’ 

Reconstruction loss: tensor(5.4467, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8726.8867) tokens processed per second.
Reconstruction loss: tensor(4.9271, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8729.6055) tokens processed per second.
Reconstruction loss: tensor(5.3757, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8732.2754) tokens processed per second.
Reconstruction loss: tensor(5.7382, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8734.8945) tokens processed per second.
Reconstruction loss: tensor(5.2772, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8737.9473) tokens processed per second.
[' ⁇ 0 ⁇ 0 ⁇ 2 ⁇ 2 ⁇ 2 ⁇ 2 ⁇ 2 ⁇ 2 ⁇ 2 ⁇ 01;  ⁇ 01;  ⁇ 01;  ⁇ 01;  ⁇ 01; n ⁇ 01;  ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; n ⁇ 01; 

Reconstruction loss: tensor(5.0748, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8727.2354) tokens processed per second.
Reconstruction loss: tensor(5.2786, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8730.0400) tokens processed per second.
Reconstruction loss: tensor(5.2780, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8732.9580) tokens processed per second.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample while batching.
Skipped overlong sample whil

Reconstruction loss: tensor(5.1128, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8727.0879) tokens processed per second.
Reconstruction loss: tensor(5.1769, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8729.7578) tokens processed per second.
Reconstruction loss: tensor(5.1107, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8732.3906) tokens processed per second.
Reconstruction loss: tensor(5.3232, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8734.8232) tokens processed per second.
Reconstruction loss: tensor(4.9217, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8737.6357) tokens processed per second.
[' ⁇ lem¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ ¬ ⁇ 

Reconstruction loss: tensor(5.5707, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8727.2959) tokens processed per second.
Reconstruction loss: tensor(5.2006, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8729.8164) tokens processed per second.
Reconstruction loss: tensor(5.2117, device='cuda:0', grad_fn=<DivBackward0>)
tensor(8732.6807) tokens processed per second.
